In [ ]:
import geopandas as gpd
import pandas as pd
import pathlib

import matplotlib.pyplot as plt

from cartopy import crs as ccrs

import os
import sys
import subprocess
import datetime
import platform
import datetime

In [ ]:
ny = gpd.read_file('BaseMap/HousingDB_by_CensusTract.shp')

In [ ]:
ny.head()

In [ ]:
ny['centract10'] = ny['centract10'].astype(str)

In [ ]:
ny['geoid'] = ny.apply(lambda x: str(x['centract10'])[:-2],axis=1)
ny.head()

In [ ]:
ny.plot()

In [ ]:
ny.crs

In [ ]:
ny = ny[['boro','geoid','geometry']]

# Exposure

In [ ]:
ex = pd.read_csv("exposure_census.csv",dtype = {"GEOID":str,"GEO_ID":str})
ex.head()

In [ ]:
ny_ex = pd.merge(ny,ex,left_on='geoid',right_on="GEO_ID",how="left")
ny_ex.head()

In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(12, 10),dpi=300)

font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

ny_ex.plot(
    column='min_max_ex',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'lower left',
                 'title': 'Exposure',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, 
                 'fontsize': 11, 
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor='white', #,np.array([0., 0., 0., 0.])
    linewidth=0.2
)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in New York - Exposure',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('NY_exposure_map.png', facecolor='w', dpi=500)

# Sensitivity

In [ ]:
sensitivity = pd.read_csv("sensitivity_census.csv", dtype={"Geocode": str})
sensitivity.head()

In [ ]:
ny_sens = pd.merge(ny,sensitivity,left_on='geoid',right_on="Geocode",how="left")
ny_sens.head()

In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(12, 10),dpi=300)

font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

ny_sens.plot(
    column='X7',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'lower left',
                 'title': 'X7',
                 'shadow': True
                },
#     missing_kwds={
#                  "color": "lightgrey",
#                  "edgecolor": "grey",
#                  "hatch": "////",
#                  "label": "Nan"
#                  },
#     edgecolor=np.array([0., 0., 0., 0.]),
    edgecolor = 'white',
    linewidth=0.2
)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in New York - Sensitivity',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('NY_X7_map.png', facecolor='w', dpi=500)

# Adaptive Capacity

In [ ]:
ac = pd.read_csv("adaptive_capacity_census.csv",dtype={'GEO_ID':str})
ac.head()

In [ ]:
ny_ac = pd.merge(ny,ac,left_on='geoid',right_on="GEO_ID",how="left")
ny_ac.head()

In [ ]:
ny_ac = gpd.GeoDataFrame(ny_ac,crs=2363)

In [ ]:
d5bemin = ny_ac['Weighted_D5be'].min()
d5bemax = ny_ac['Weighted_D5be'].max()
ny_ac['D5be_score'] = ny_ac.apply(lambda x: (x['Weighted_D5be']-d5bemin)/(d5bemax-d5bemin),axis=1)

In [ ]:
d4cmin = ny_ac['Weighted_D4c'].min()
d4cmax = ny_ac['Weighted_D4c'].max()
ny_ac['D4c_score'] = ny_ac.apply(lambda x: (x['Weighted_D4c']-d4cmin)/(d4cmax-d4cmin),axis=1)

In [ ]:
d4amin = ny_ac['Weighted_D4a'].min()
d4amax = ny_ac['Weighted_D4a'].max()
ny_ac['D4a_score'] = ny_ac.apply(lambda x: (x['Weighted_D4a']-d4amin)/(d4amax-d4amin),axis=1)

In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(12, 10),dpi=300)

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

ny_ac.plot(
    column='D5be_score',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': '',
                 'shadow': True
                },
    missing_kwds={
                 "color": "lightgrey",
                 "edgecolor": "grey",
                 "hatch": "////",
                 "label": "Nan"
                 },
#     edgecolor=np.array([0., 0., 0., 0.]),
    edgecolor = 'white',
    linewidth=0.2
)

ax.axis('off')
ax.set_title('Working-population within 45-minute transit commute', fontsize=15,pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('D5be_normal_map.png', facecolor='w', dpi=500)

In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(12, 10),dpi=300)

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

ny_ac.plot(
    column='D4c_score',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': '',
                 'shadow': True
                },
    missing_kwds={
                 "color": "lightgrey",
                 "edgecolor": "grey",
                 "hatch": "////",
                 "label": "Nan"
                 },
#     edgecolor=np.array([0., 0., 0., 0.]),
    edgecolor = 'white',
    linewidth=0.2
)

ax.axis('off')
ax.set_title('Aggregate frequency of transit service within 0.25 miles \n of block group boundary per hour during evening peak period', fontsize=15,pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('D4c_normal_map.png', facecolor='w', dpi=500)

In [ ]:


import numpy as np

fig, ax = plt.subplots(figsize=(12, 10),dpi=300)

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

ny_ac.plot(
    column='D4a_score',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': '',
                 'shadow': True
                },
    missing_kwds={
                 "color": "lightgrey",
                 "edgecolor": "grey",
                 "hatch": "////",
                 "label": "Nan"
                 },
#     edgecolor=np.array([0., 0., 0., 0.]),
    edgecolor = 'white',
    linewidth=0.2
)

ax.axis('off')
ax.set_title('Distance from population weighted centroid to nearest transit stop', fontsize=15,pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('D4a_normal_map.png', facecolor='w', dpi=500)

In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(12, 10),dpi=300)

font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

ny_ac.plot(
    column='min_max_ac',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'lower left',
                 'title': 'Adaptive Capacity',
                 'shadow': True
                },
    missing_kwds={
                 "color": "lightgrey",
                 "edgecolor": "grey",
                 "hatch": "////",
                 "label": "Nan"
                 },
#     edgecolor=np.array([0., 0., 0., 0.]),
    edgecolor = 'white',
    linewidth=0.2
)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in New York - Adaptive Capacity',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)

ax.set_aspect('equal','box')
fig.tight_layout()

fig.savefig('NY_AC_map.png', facecolor='w', dpi=500)

# Vulnerability Score

In [ ]:
vc = pd.read_csv('Vulnerability Score.csv',dtype={'GEOID':str})
vc.head()

In [ ]:
ny_vc = pd.merge(ny,vc,left_on='geoid',right_on="GEOID",how="left")
ny_vc.head()

In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(10, 10),dpi=300)

font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

ny_vc.plot(
    column='vul_score_mulpi',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     scheme='UserDefined',
#     classification_kwds={
#                         'bins': [0, 0.05, 0.1, 0.15, 0.2, 0.25]
#                         },
#     k=7,
    legend_kwds={'loc': 'center left',
                 'title': 'Vulnerability Score',
                 "fmt": "{:.3f}",
                 
                 'title_fontsize': 14, 
                 'fontsize': 14, 
                 'shadow': True},
#     missing_kwds={
#                  "color": "lightgrey",
#                  "edgecolor": "grey",
#                  "hatch": "////",
#                  "label": "Nan"
#                  },
#     edgecolor=np.array([0., 0., 0., 0.]),
    edgecolor = 'white',
    linewidth=0.2,
#     vmin=vmin, vmax=vmax
)

# ax.axis('on')
plt.xticks([])
plt.yticks([])
ax.set_title('Multiplicative Transportation Energy Vulnerability in New York', fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('NY_vulnerability_mulpi.png', facecolor='w', dpi=500)

In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(10, 10),dpi=300)

font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

ny_vc.plot(
    column='vul_score_add',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     scheme='UserDefined',
#     classification_kwds={
#                         'bins': [0, 0.05, 0.1, 0.15, 0.2, 0.25]
#                         },
#     k=7,
    legend_kwds={'loc': 'center left',
                 'title': 'Vulnerability Score',
                 "fmt": "{:.3f}",
                 
                 'title_fontsize': 14, 
                 'fontsize': 14, 
                 'shadow': True},
#     missing_kwds={
#                  "color": "lightgrey",
#                  "edgecolor": "grey",
#                  "hatch": "////",
#                  "label": "Nan"
#                  },
#     edgecolor=np.array([0., 0., 0., 0.]),
    edgecolor = 'white',
    linewidth=0.2,
#     vmin=vmin, vmax=vmax
)

# ax.axis('on')
plt.xticks([])
plt.yticks([])
ax.set_title('Additive Transportation Energy Vulnerability in New York', fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('NY_vulnerability_add.png', facecolor='w', dpi=500)

In [ ]:
ny_ex = ny_ex[['geoid','NAME','min_max_ex']]
ny_sens = ny_sens[['geoid','min_max_sens']]
ny_ac = ny_ac[['geoid','min_max_ac']]
ny_vc = ny_vc[['geoid','vul_score_mulpi','vul_score_add']]
ny_all_census  = pd.merge(ny_ex, ny_sens, on='geoid')
ny_all_census  = pd.merge(ny_all_census,ny_ac, on='geoid')
ny_all_census  = pd.merge(ny_all_census,ny_vc, on='geoid')

In [ ]:
ny_all_census.to_csv('ny_all_census.csv')